# TP1 - Finding Keys using Functional Dependencies 
--------------------------

In this lab, we are going to learn 

- how to use Jupyter
- how to use SQLite
- how to discover keys in relations

## How to use Jupyter

To execute a cell, click on it, and then use SHIFT+ENTER (this will execute the contents of the cell, and move down to the next one!)

To edit a cell, click on it. If the cell uses markdown code, then use ENTER to edit it.

See the Help menu for other useful keyboard commands. You can always use the menu bar instead as well.


In [1]:
print("Hello world!")

Hello world!


Another example:

In [2]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


#### Exercise 1

Print numbers 1 to 9 in reverse order

In [4]:
for i in range(9,0,-1):
    print(i)

9
8
7
6
5
4
3
2
1


## How to use SQLite

To work with SQL easily in a notebook, we'll load the ipython-sql extension as follows:

In [ ]:
#KO
#!pip install pysqlite3 

#OK, à ne pas relancer ?     
#!conda install -c conda-forge ipython-sql -y

Solving environment: done

## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs: 
    - ipython-sql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wheel-0.32.1               |           py36_0          34 KB  conda-forge
    prettytable-0.7.2          |             py_2          14 KB  conda-forge
    ipython_genutils-0.2.0     |             py_1          21 KB  conda-forge
    pygments-2.2.0             |             py_1         622 KB  conda-forge
    six-1.11.0                 |        py36_1001          21 KB  conda-forge
    setuptools-40.4.3          |           py36_0         554 KB  conda-forge
    traitlets-4.3.2            |        py36_1000         131 KB  conda-forge
    parso-0.3.1                |             py_0          59 KB  conda-forge
    pip-18.1                   |        py36_1000         1.8 MB  conda-forge
    opens

In [13]:
%load_ext sql
%sql sqlite://

'Connected: @None'

We are going to create a table:

In [14]:
%%sql DROP TABLE IF EXISTS T;
CREATE TABLE T(course VARCHAR, classroom INT, time INT);
INSERT INTO T VALUES ('CS 364', 132, 900);
INSERT INTO T VALUES ('CS 245', 140, 1000);
INSERT INTO T VALUES ('EE 101', 210, 900);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Let's run our first SQL query:

In [15]:
%sql SELECT * FROM T;

 * sqlite://
Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900


#### Exercise 2

List the name of the courses with time less than 950

In [16]:
%sql SELECT * FROM T WHERE time <= 950;

 * sqlite://
Done.


course,classroom,time
CS 364,132,900
EE 101,210,900


## How to discover keys in relations

Now, we are going to work with functional dependencies, keys and closures. Our final goal is going to build a method to find keys in a relation.

### Functional Dependencies

Recall that given a set of attributes  $\{A_1, \dots, A_n\}$ and a set of FDs $\Gamma$

The closure, denoted $\{A_1, \dots, A_n\}^+$, is defined to be the largest set of attributes B s.t. $$A_1,\dots,A_n \rightarrow B \text{ using } \Gamma.$$

We're going to use some functions to compute the closure of a set of attributes and other such operations (_from CS145 Stanford_)

In [17]:
# Source code

def to_set(x):
  """Convert input int, string, list, tuple, set -> set"""
  if type(x) == set:
    return x
  elif type(x) in [list, set]:
    return set(x)
  elif type(x) in [str, int]:
    return set([x])
  else:
    raise Exception("Unrecognized type.")

def fd_to_str(lr_tuple):
    lhs = lr_tuple[0]
    rhs = lr_tuple[1]
    return ",".join(to_set(lhs)) + " -> " + ",".join(to_set(rhs))

def fds_to_str(fds): return "\n\t".join(map(fd_to_str, fds))

def set_to_str(x): return "{" + ",".join(x) + "}"

def fd_applies_to(fd, x): 
  lhs, rhs = map(to_set, fd)
  return lhs.issubset(x)

def print_setup(A, fds):
  print("Attributes = " + set_to_str(A))
  print("FDs = \t" + fds_to_str(fds))

def print_fds(fds):
  print("FDs = \t" + fds_to_str(fds))    


Now, let's look at a concrete example. For example, the code for

attributes = { name, category, color, department, price}

and functional dependencies:

name $\rightarrow$ color

category $\rightarrow$ department

color, category $\rightarrow$ price

is the following:

In [18]:
attributes  = set(["name", "category", "color", "department", "price"]) # These are the attribute set.
fds = [(set(["name"]),"color"),
         (set(["category"]), "department"),
         (set(["color", "category"]), "price")]

print_setup(attributes, fds)

Attributes = {price,name,category,color,department}
FDs = 	name -> color
	category -> department
	category,color -> price


### Closure of a set of Attributes

Let's implement the algorithm for obtaining the closure of a set of attributes:

In [19]:
def compute_closure(x, fds, verbose=False):
    bChanged = True        # We will repeat until there are no changes.
    x_ret    = x.copy()    # Make a copy of the input to hold x^{+}
    while bChanged:
        bChanged = False   # Must change on each iteration
        for fd in fds:     # loop through all the FDs.
            (lhs, rhs) = map(to_set, fd) # recall: lhs -> rhs
            if fd_applies_to(fd, x_ret) and not rhs.issubset(x_ret):
                x_ret = x_ret.union(rhs)
                if verbose:
                    print("Using FD " + fd_to_str(fd))
                    print("\t Updated x to " + set_to_str(x_ret))
                bChanged = True
    return x_ret

As an example, let's compute the closure for the attribute "name":

In [20]:
A = set(["name"])
compute_closure(A,fds, True)

Using FD name -> color
	 Updated x to {name,color}


{'color', 'name'}

#### Exercise 3

Is the attribute "name" a superkey for this relation? Why?

### Keys and Superkeys

Next, we'll add some new functions now for finding superkeys and keys.  Recall:
* A _superkey_ for a relation $R(B_1,\dots,B_m)$ is a set of attributes $\{A_1,\dots,A_n\}$ s.t.
$$ \{A_1,\dots,A_n\} \rightarrow B_{j} \text{ for all } j=1,\dots m$$
* A _key_ is a minimal (setwise) _superkey_

The algorithm to determine if a set of attributes $A$ is a superkey for $X$ is actually very simple (check if $A^+=X$):

In [21]:
def is_superkey_for(A, X, fds, verbose=False): 
    return X.issubset(compute_closure(A, fds, verbose=verbose))

Is "name" a superkey of the relation?

In [22]:
is_superkey_for(A, attributes, fds)

False

Then, to check if $A$ is a key for $X$, we just confirm that:
* (a) it is a superkey
* (b) there are no smaller superkeys (_Note that we only need to check for superkeys of one size smaller_)

In [23]:
import itertools
def is_key_for(A, X, fds, verbose=False):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey_for(A, X, fds) and \
        all([not is_superkey_for(set(SA), X, fds) for SA in subsets])

Now, let's look at another example:

attributes = { cru, type, client, remise}

and functional dependencies:

cru $\rightarrow$ type

type, client $\rightarrow$ remise

#### Exercise 4

Is "cru" and "client" a key of the relation? Why?

In [25]:
attributes  = set(["cru", "type", "client", "remise"]) # These are the attribute set.
fds = [(set(["cru"]),"type"),
         (set(["type", "client"]), "remise")]
is_key_for(set(["cru", "client"]), attributes, fds)
#cru gives type, + client gives remise, full cover

True

Because it's a superkey and it is minimal. 

### Closure of a set of functional dependencies

The algorithm to find the closure of a set of functional dependencies is the following:

In [26]:
import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))
def closure(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size) 
        for SA in subsets:      # loop through all the subsets.
            cl = compute_closure(set(SA), fds, verbose)
            if len(cl.difference(SA)) > 0: 
                c.extend([(set(SA), cl.difference(SA))])
    return c

Let's see some examples of how to use it:

In [27]:
attributes1 = set(['A', 'B', 'C', 'D'])
fds1 = [(set(['A', 'B']), 'C'),
     (set(['A', 'D']), 'B'),
     (set(['B']), 'D')]
print_fds(closure(attributes1, fds1))


FDs = 	B -> D
	C,B -> D
	A,D -> C,B
	B,A -> D,C
	C,A,D -> B
	C,B,A -> D
	B,A,D -> C


In [28]:
attributes2 = set (['CRU', 'TYPE', 'CLIENT', 'REMISE'])
fds2 = [(set(['CRU']), 'TYPE'),
     (set(['TYPE', 'CLIENT']), 'REMISE')]
print_fds(closure(attributes2, fds2))

FDs = 	CRU -> TYPE
	TYPE,CLIENT -> REMISE
	CRU,CLIENT -> REMISE,TYPE
	CRU,REMISE -> TYPE
	CRU,TYPE,CLIENT -> REMISE
	CRU,REMISE,CLIENT -> TYPE


In [29]:
attributes3 = set( ['N VEH', 'TYPE', 'COULEUR', 'MARQUE', 'PUISSANCE'])
fds3 = [(set(['N VEH']), 'TYPE'), 
      (set(['N VEH']), 'COULEUR'),
      (set(['TYPE']), 'MARQUE'),
      (set(['TYPE']), 'PUISSANCE')]
print_fds(closure(attributes3,fds3))

FDs = 	TYPE -> PUISSANCE,MARQUE
	N VEH -> PUISSANCE,TYPE,MARQUE,COULEUR
	N VEH,TYPE -> PUISSANCE,MARQUE,COULEUR
	PUISSANCE,TYPE -> MARQUE
	N VEH,MARQUE -> PUISSANCE,TYPE,COULEUR
	N VEH,COULEUR -> PUISSANCE,TYPE,MARQUE
	TYPE,COULEUR -> PUISSANCE,MARQUE
	PUISSANCE,N VEH -> TYPE,MARQUE,COULEUR
	MARQUE,TYPE -> PUISSANCE
	PUISSANCE,N VEH,MARQUE -> TYPE,COULEUR
	PUISSANCE,TYPE,COULEUR -> MARQUE
	MARQUE,TYPE,COULEUR -> PUISSANCE
	N VEH,MARQUE,COULEUR -> PUISSANCE,TYPE
	N VEH,MARQUE,TYPE -> PUISSANCE,COULEUR
	N VEH,TYPE,COULEUR -> PUISSANCE,MARQUE
	PUISSANCE,N VEH,TYPE -> MARQUE,COULEUR
	PUISSANCE,N VEH,COULEUR -> TYPE,MARQUE
	N VEH,MARQUE,TYPE,COULEUR -> PUISSANCE
	PUISSANCE,N VEH,MARQUE,COULEUR -> TYPE
	PUISSANCE,N VEH,MARQUE,TYPE -> COULEUR
	PUISSANCE,N VEH,TYPE,COULEUR -> MARQUE


Now, let's write a method to find superkeys of the relations:


In [30]:
def superkeys(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size)
        for SA in subsets:
            cl = compute_closure(set(SA), fds, verbose)
            if cl == X and len(cl.difference(SA)) > 0: ## cl = X
                c.extend([SA])
    return c

In [31]:
superkeys(attributes1, fds1)


[('A', 'D'), ('B', 'A'), ('C', 'A', 'D'), ('C', 'B', 'A'), ('B', 'A', 'D')]

Let's see some examples:

In [32]:
superkeys(attributes2, fds2)


[('CRU', 'CLIENT'), ('CRU', 'TYPE', 'CLIENT'), ('CRU', 'REMISE', 'CLIENT')]

In [33]:
superkeys(attributes3, fds3)

[('N VEH',),
 ('TYPE', 'N VEH'),
 ('MARQUE', 'N VEH'),
 ('N VEH', 'COULEUR'),
 ('PUISSANCE', 'N VEH'),
 ('MARQUE', 'PUISSANCE', 'N VEH'),
 ('MARQUE', 'N VEH', 'COULEUR'),
 ('TYPE', 'MARQUE', 'N VEH'),
 ('TYPE', 'N VEH', 'COULEUR'),
 ('TYPE', 'PUISSANCE', 'N VEH'),
 ('PUISSANCE', 'N VEH', 'COULEUR'),
 ('TYPE', 'MARQUE', 'N VEH', 'COULEUR'),
 ('MARQUE', 'PUISSANCE', 'N VEH', 'COULEUR'),
 ('TYPE', 'MARQUE', 'PUISSANCE', 'N VEH'),
 ('TYPE', 'PUISSANCE', 'N VEH', 'COULEUR')]

#### Exercise 5

Implement a `keys` method to find keys of a relation.

**Note**: If there exist multiple keys of a relation, the `keys` method should return at least one of them.

In [102]:
def keys(X, fds):
    c = superkeys(X, fds)
    c = list(map(lambda x: set(list(x)), c))
    d = c.copy();
    for elt in c:
        for other in c:
            if elt != other and elt.issubset(other):
                if other in d:
                    d.remove(other)
    return d

Test it 

In [103]:
a = {'CRU', 'CLIENT'}
b = {'CRU', 'REMISE', 'CLIENT'}
a.issubset(b)


True

In [104]:
keys(attributes1, fds1)

[{'A', 'D'}, {'A', 'B'}]

In [105]:
keys(attributes2, fds2)

[{'CLIENT', 'CRU'}]

In [106]:
keys(attributes3, fds3)

[{'N VEH'}]